# introduction

The Black-Litterman (BL) asset allocation model is pretty amazing. It consists of three brilliant ideas, whose interplay is a brilliant idea on its own. It is (1) a Bayesian econometrics model (2) with a very smart choice of the prior and the data, and (3) an asset allocation model. Neither of the ideas feels out of place, and neither was introduced just for the sake of it.

In this notebook, I would like to visualize the Bayesian dimension of Black-Litterman that was pointed out (among others) by [Kolm and Ritter (2017)](https://cims.nyu.edu/~ritter/kolm2017bayesian.pdf). I will use a library for Bayesian inference &ndash; `Turing.jl` &ndash; to numerically solve the example in [He and Litterman (1999)](https://papers.ssrn.com/sol3/papers.cfm?abstract_id=334304), and, of course, will arrive at the same values as in the closed-form solution derived by the original inventors.

# model recap
Black and Litterman start by pondering how a portfolio manager could incorporate (possibly contradicting) views about expected returns (e.g from different analysts) into the mean-variance optimization problem. Neat! In a nutshell, here is the answer: 
- treat the expected returns as a random variable; 
- make the views are a (conveniently linear) function of that random variable plus some noise; 
- assume the Normal distribution whenever possible;
- suddenly, the linearity and the Normal distribution allows to calculate the mean and variance of asset returns...
- ...which the portfolio manager can plug into the mean-variance optimization.

To put future academics further to shame, Black and Litterman add market equilibrium and currency hedging to the model.

# black-litterman model through bayesian lens
A distribution parameter is a random variable, you say? Let's assume a prior for it, and the Bayesian connection is palpable &ndash; but what is the data here that can be used to update the prior? The views are! Kolm and Ritter (2017) write:

> we suggest thinking of a portfolio manager's forecast as an “observation of the future” in which the measuring device is a rather murky and unreliable crystal ball. Only in this way is it analogous to the noisy measurements in experimental design which much of statistics is designed to model.

The views can be just one view, they can be contradicting views, they can be a function of the above random variable, etc. When the prior is updated, the mean and variance of the posterior are used in the Markowitz optimization.

Black and Litterman choose the prior super cleverly: they say that without any views, the result of the optimization should be the market portfolio, in which the assets are weighted according to their capitalization (this is tantamount to saying that CAPM holds perfectly). 

Anyway, here is the model in detail:

1. To perform the Markowitz optimization, the expected value $\mu_r$ and the covariance matrix $\Sigma_r$ of asset returns are needed;
2. $\Sigma_r$ is assumed to be known or at least estimated with very high precision;
3. $\mu_r$, however, are a vector-valued random variable, initially believed to follow the (multivariate) Normal distribution with mean $\Pi$ and covariance $\Sigma_{\mu} = \tau \Sigma_r$; $\Pi$ is taken to be the expected returns which would without any other information result in the market portfolio being mean-variance efficient; in other words, given $\Pi$, $w_m$ is the solution to: 
    $$
        \underset{w}{\max} \ w'\Pi - \frac{\delta}{2} w'\Sigma_r w
    $$
    for some value of $\delta$.
4. the views are data points which can tell a little more about the distribution of $\mu_r$; the views can be direct or indirect, but always possible to express as 
    $$
        P \mu_r = q + \varepsilon_q,
    $$ 
    where $\varepsilon$ is a random variable introducing the noise (of course, Normally distributed!)
5. finally, the update of the prior belief from (3) is made based on the views in the Bayesian fashion: intuitively, the updated distribution will reflect the views, the stronger so, the more numerous and less noisy they are;
6. finally, we don't need the whole updated distribution of $\mu_r$, but only its mean because, nesting Normal random variables preserves the mean of the innermost variable, and because, bar the covariance, _nothing else matters_ for the mean-variance optimization.

Black and Litterman provide a closed-form solution for the mean of the asset returns (which turns out to be the mean of the updated distribution of $\mu_r$) and their covariance (which turns out to be updated, too). The solution is derived from some simple mathematics without references to prior, conjugates, posterior, likelihoods and all that Bayesian stuff...

# implementation as a bayesian update problem
...but where is fun is that? Let's solve the Bayesian problem numerically using the quite amazing `Turing.jl` library, and confirm what I claim above!

First, recreate the environment, where `Turing` and other necessary packages are installed, and import them.

In [1]:
using Pkg

Pkg.activate(".")

  Activating project at `~/projects/black-litterman-bayes`


In [2]:
using Turing, Distributions, MCMCChains

ERROR: Method overwriting is not permitted during Module precompilation. Use `__precompile__(false)` to opt-out of precompilation.


Then, let's import some helper function, like the ones fetching the data from He and Litterman (1999).

In [3]:
# datafeed functions and nearest_pd_matrix
include("src_julia/datafeed.jl")
include("src_julia/utilities.jl")

nearest_pd_matrix (generic function with 1 method)

Now, let's load data from that paper: the covariance matrix is constructed using the correlation structure from Table 1 and the standard deviations from Table 2; the market portfolio weights are from Table 2.

In [4]:
# covariance and market weights
Σ_r = get_covariance();

# vcv can be numerically non-hermitian, so let's fix it
Σ_r = nearest_pd_matrix(Σ_r);

# Pi, the mean of expected returns' distribution
Π = get_table(2)[:, :Pi] / 100;

n_assets = length(Π);

println("covariance:"); flush(stdout)
display(round.(Σ_r, digits=4))

covariance:


7×7 Matrix{Float64}:
 0.0256  0.0159  0.019   0.0223  0.0148  0.0164  0.0147
 0.0159  0.0412  0.0334  0.036   0.0132  0.0247  0.0296
 0.019   0.0334  0.0615  0.0579  0.0185  0.0388  0.031
 0.0223  0.036   0.0579  0.0734  0.0201  0.0421  0.0331
 0.0148  0.0132  0.0185  0.0201  0.0441  0.017   0.012
 0.0164  0.0247  0.0388  0.0421  0.017   0.04    0.0244
 0.0147  0.0296  0.031   0.0331  0.012   0.0244  0.035

Also, define parameters as in the paper: 
  - the market risk aversion $\delta$ set to 0.025;
  - the scaling factor $\tau$ for the covariance in the prior distribution of the $\mu_r$ is set to 0.05; 
  - the variance of the view about the German market $\omega$ is set via $\omega/\tau = 0.021$.

In [5]:
# market risk aversion
δ = 2.5;

# variance scaling for the expected returns prior
τ = 0.05;

# (relative) view uncertainty
omega_through_tau = 0.021;

# view uncertainty
ω = omega_through_tau * τ;

Finally, define the necessary $P$ and $Q$ matrices for the view about the German market outperforming the rest of Europe by 5%; $P$ is taken from Table 4. Since we are dealing with the single view, it's easier to define $P$ and $Q$ as vectors:

In [6]:
# view that ger outperformce market cap-weighted fra+gbr by 5% (from table 3)
p = get_table(4)[:, :p] / 100;
q = 0.05;

println("long-short portfolio with the view on german market:")
println(p)

long-short portfolio with the view on german market:
[0.0, 0.0, -0.295, 1.0, 0.0, -0.705, 0.0]


Now, let's set up a Bayesian model by:

1. treating the expected returns of the assets as a random variable with a prior distribution;
2. treating the view about the German market as the realization of a random variable connected to the expected returns;
3. adjusting the distribution of returns.

$$
\begin{gather}
    \mu_r \sim N(\Pi, \tau \Sigma_r) \\
    q | \mu_r \sim N(P'\mu_r, \omega) \\
    r | \mu \sim N(\mu_r, \Sigma),
\end{gather}
$$

where (1) defines the prior distribution of the expected returns means that the expected returns are initially

In [7]:
# black-litterman bayesian model
@model function blb(_q)

    global Π, Σ_r, p, ω

    # prior on mu
    μ_r ~ MvNormal(Π, τ* Σ_r);

    # given mu, the views are distributed as:
    _q ~ Normal(dot(p, μ_r), sqrt(ω));
    
    # condition the model on data
    return _q
end

blb (generic function with 2 methods)

...and sample from the posterior:

In [10]:
chain = sample(blb(q), NUTS(), 40000, progress = false);

┌ Info: Found initial step size
│   ϵ = 0.025
└ @ Turing.Inference /home/ipozdeev/.julia/packages/Turing/JVSRF/src/mcmc/hmc.jl:191


In [9]:
post_est, _ = describe(chain);

μ_bar = round.(post_est[:, :mean] * 100, digits=1);

μ_bar_exog = get_table(4)[:, :mu_bar];
print(DataFrame(:μ_bar => μ_bar, :μ_bar_paper => μ_bar_exog))

7×2 DataFrame
 Row │ μ_bar    μ_bar_paper 
     │ Float64  Float64     
─────┼──────────────────────
   1 │     4.4          4.3
   2 │     7.7          7.6
   3 │     9.5          9.3
   4 │    11.2         11.0
   5 │     4.6          4.5
   6 │     7.1          7.0
   7 │     8.2          8.1

Bar numerical differences, a perfect match! Now, what about the covariance $\Sigma_r$? To get to it, we need to sample from the posterior of $r$, which can be achieved by rewriting the function slightly (see the [Turing reference](https://turing.ml/dev/docs/using-turing/guide#treating-observations-as-random-variables) for details):

In [13]:
# black-litterman bayesian model
@model function blb_with_r(q, _r)

    global Π, Σ_r, p, ω

    if _r === missing
        # Initialize `q` if missing
        _r = Vector{Float64}(undef, n_assets);
    end

    # prior on mu
    μ_r ~ MvNormal(Π, τ* Σ_r);

    # given mu, the views are distributed as:
    _q ~ Normal(dot(p, μ_r), sqrt(ω));
    
    # given mu, the returns are distributed as:
    _r ~ MvNormal(μ_r, Σ_r);
    
    # condition the model on data
    return q
end

blb_with_r (generic function with 2 methods)

In [14]:
# sample from the posterior of r
chain = sample(blb_with_r(q, missing), NUTS(), 40000, progress = false);

┌ Info: Found initial step size
│   ϵ = 0.0125
└ @ Turing.Inference /home/ipozdeev/.julia/packages/Turing/JVSRF/src/mcmc/hmc.jl:191


In [20]:
r_sim = DataFrame(chain)[!, (n_assets+1+3):(n_assets+1+3+n_assets-1)];

print(r_sim[1:6, :])

6×7 DataFrame
 Row │ _r[1]       _r[2]       _r[3]       _r[4]       _r[5]       _r[6]       _r[7]      
     │ Float64     Float64     Float64     Float64     Float64     Float64     Float64    
─────┼────────────────────────────────────────────────────────────────────────────────────
   1 │ -0.278585    0.0476169   0.177344   -0.0659182  -0.228496   -0.0427864   0.0507353
   2 │ -0.0997189   0.119443   -0.126084   -0.0130549  -0.0951963  -0.28269     0.0738249
   3 │ -0.0381792   0.10803     0.0269905  -0.113583   -0.121041   -0.18265     0.0481404
   4 │ -0.278518   -0.199939   -0.200783   -0.321998    0.184968    0.0608774  -0.0611563
   5 │ -0.1367      0.0521286   0.0501448   0.155181    0.168805    0.0724417  -0.0674361
   6 │  0.0734958   0.433291    0.229726    0.121449    0.202212    0.307832    0.607469

In [ ]:
Now we can just estimate the covariance matrix of these values.

In [24]:
Σ_r_post = cov(Matrix(r_sim));

display(round.(Σ_r_post, digits=4))

7×7 Matrix{Float64}:
 0.0269  0.0167  0.02    0.0236  0.0154  0.0172  0.0153
 0.0167  0.0431  0.035   0.0378  0.0139  0.0259  0.031
 0.02    0.035   0.0647  0.0608  0.0196  0.0412  0.0328
 0.0236  0.0378  0.0608  0.0773  0.0213  0.0446  0.035
 0.0154  0.0139  0.0196  0.0213  0.0457  0.0179  0.0128
 0.0172  0.0259  0.0412  0.0446  0.0179  0.0422  0.0257
 0.0153  0.031   0.0328  0.035   0.0128  0.0257  0.0367

Compared to the original covariance matrix, the posterior is larger in magnitude

In [25]:
display(round.(Σ_r, digits=4))

7×7 Matrix{Float64}:
 0.0256  0.0159  0.019   0.0223  0.0148  0.0164  0.0147
 0.0159  0.0412  0.0334  0.036   0.0132  0.0247  0.0296
 0.019   0.0334  0.0615  0.0579  0.0185  0.0388  0.031
 0.0223  0.036   0.0579  0.0734  0.0201  0.0421  0.0331
 0.0148  0.0132  0.0185  0.0201  0.0441  0.017   0.012
 0.0164  0.0247  0.0388  0.0421  0.017   0.04    0.0244
 0.0147  0.0296  0.031   0.0331  0.012   0.0244  0.035

...by &ndash; surprize? &ndash; $(1 + \tau)$!

In [26]:
display(round.(Σ_r_post ./ Σ_r, digits=2))

7×7 Matrix{Float64}:
 1.05  1.05  1.05  1.05  1.04  1.05  1.04
 1.05  1.05  1.05  1.05  1.05  1.05  1.05
 1.05  1.05  1.05  1.05  1.06  1.06  1.06
 1.05  1.05  1.05  1.05  1.06  1.06  1.06
 1.04  1.05  1.06  1.06  1.04  1.05  1.06
 1.05  1.05  1.06  1.06  1.05  1.06  1.05
 1.04  1.05  1.06  1.06  1.06  1.05  1.05